**Audio Dataset Evaluation**


This notebook contains the evaluation of a german medical dataset with various models:
1. Salad Transcription
2. Assembly AI
3. Whisper Large (v2)
4. AWS Transcribe

And evaluation metrics.

---

In [1]:
!pip install pydub

In [2]:
!pip install requests

In [3]:
! pip install numpy

In [4]:
!pip install pandas

In [5]:
!pip install openpyxl


In [6]:
!pip install ffmpeg-python


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
path = "/content/drive/MyDrive/Audio to text"
os.chdir(path)
os.getcwd()

'/content/drive/MyDrive/Audio to text'

In [9]:
# load packages
import requests
import json
import base64
from pydub import AudioSegment
import numpy as np
import pandas as pd
import whisper

In [10]:
# load excel file ( ground truth)
os.getcwd()
df = pd.read_excel("Metadata.xlsx")

In [11]:
df.head()

,Sr No.,File name,Duration,User Age,Gender,Text File,Domain,Device Details
0,1,German_Healthcare_1,00:00:07,37,Male,Manche Krankheiten treten gehäuft zu bestimmte...,Healthcare,"12, samsung, SM-A515F"
1,2,German_Healthcare_2,00:00:06,37,Male,Die Arthrose ist eine schmerzhafte Gelenk-Erkr...,Healthcare,"12, samsung, SM-A515F"
2,3,German_Healthcare_3,00:00:06,37,Male,"Arthrose entsteht, wenn der Gelenk-Knorpel irr...",Healthcare,"12, samsung, SM-A515F"
3,4,German_Healthcare_4,00:00:09,26,Male,Stress spielt bei der Entstehung einer Depress...,Healthcare,"12, OnePlus, DN2103"
4,5,German_Healthcare_5,00:00:09,26,Male,Schwere und chronische Krankheiten sind zudem ...,Healthcare,"12, OnePlus, DN2103"


In [12]:
text = ' '.join(df['Text File'].values)
text

'Manche Krankheiten treten gehäuft zu bestimmten Jahreszeiten auf. Die Arthrose ist eine schmerzhafte Gelenk-Erkrankung. Arthrose entsteht, wenn der Gelenk-Knorpel irreparabel geschädigt wird. Stress spielt bei der Entstehung einer Depression eine entscheidende Rolle. Schwere und chronische Krankheiten sind zudem eine Dauerbelastung für die Psyche. Diabetes mellitus ist eine krankhafte Störung des Zuckerstoffwechsels. Körperliche Gesundheit bedeutet, dass der körperliche Zustand eines Menschen keinen negativen Einflüssen, Einschränkungen oder Verletzungen unterliegt. Psychische Gesundheit ist ein Zustand, in dem ein Mensch keinen seelischen und geistigen Einschränkungen oder Störungen unterliegt. Eine Erkältung – oder auch grippaler Infekt – ist eine Erkrankung der oberen Atemwege. Zwischen der Ansteckung und dem Krankheitsbeginn liegen üblicherweise zwölf Stunden bis drei Tage. Eine Erkältung beginnt in vielen Fällen zunächst mit Halsschmerzen. Eine unkomplizierte Erkältung dauert nor

In [13]:
print(f"The length of the transcript is: {len(text)} characters")

The length of the transcript is: 3047 characters


## Audio transcription

In [14]:
# put audios into one file and export
input_folder = "./FB_PROMPT_HC_GERMAN"
output_file = "./FB_PROMPT_HC_GERMAN/german_audio_full"

# Get a list of all .wav files in the folder
wav_files = [f for f in os.listdir(input_folder) if f.endswith(".wav")]
wav_files.sort()

combined_audio = AudioSegment.silent(duration=0)

# Loop through each .wav file and append it to the combined audio
for wav_file in wav_files:
    file_path = os.path.join(input_folder, wav_file)
    audio = AudioSegment.from_wav(file_path)
    combined_audio += audio  # Concatenate audio files

# Export the final combined file as MP3
combined_audio.export(output_file, format="mp3")
print(f"Combined audio saved to: {output_file}")

Combined audio saved to: ./FB_PROMPT_HC_GERMAN/german_audio_full


Salad

*This code was given by Salad Transription Serivces, you can find more on their website.*

In [25]:
def send_transcription_request(audio_url, organization_name, salad_key, language_code="en", input_params=None):
    """
    Sends a transcription request to the Salad Transcription API with dynamic input parameters.

    Args:
        audio_url (str): The signed Salad Storage URL for the audio file.
        api_endpoint (str): The Salad Transcription API endpoint.
        salad_key (str): The Salad API key for authentication.
        language_code (str): The language of the audio file (default: "en").
        input_params (dict): Additional input parameters to include in the request.

    Returns:
        dict: The response from the API, including the job ID.
    """
    # Set headers
    headers = {
        "Salad-Api-Key": salad_key,
        "Content-Type": "application/json",
        "accept": "application/json",
    }
    api_endpoint = f"https://api.salad.com/api/public/organizations/{organization_name}/inference-endpoints/transcribe/jobs"
    # Prepare the input dictionary, appending the audio URL and language code
    input_data = input_params.copy() if input_params else {}
    input_data["url"] = audio_url
    input_data["language_code"] = language_code

    # Request body
    data = {
        "input": input_data
    }

    # Send the request
    response = requests.post(api_endpoint, headers=headers, json=data)
    response.raise_for_status()  # Raise an error for bad responses

    # Extract and return the response
    return response.json()

In [27]:
def get_transcription_result(job_id, salad_key, organization_name):
    """
    Retrieves the transcription result from the Salad Transcription API for a given job ID.

    Args:
        job_id (str): The unique identifier of the transcription job.
        salad_key (str): The Salad API key for authentication.
        organization_name (str): Name of the Salad organization.

    Returns:
        dict: The response from the API containing the job status and transcription result.
    """
    # Construct the Salad API endpoint for retrieving job results
    api_endpoint = f"https://api.salad.com/api/public/organizations/{organization_name}/inference-endpoints/transcribe/jobs/{job_id}"

    # Set headers
    headers = {
        "Salad-Api-Key": salad_key,
        "Content-Type": "application/json",
        "accept": "application/json",
    }

    # Send the request
    response = requests.get(api_endpoint, headers=headers)
    response.raise_for_status()

    # Extract and return the response
    return response.json()

In [29]:
salad_transcript = " Manche Krankheiten treten gehäuft zu bestimmten Jahreszeiten auf. Zwischen der Ansteckung und dem Krankheitsbeginn liegen üblicherweise 12 Stunden bis 3 Tage. Eine Erkältung beginnt in vielen Fällen zunächst mit Halsschmerzen. Eine unkomplizierte Erkältung dauert normalerweise 9 Tage, bei Kindern bis zu 15 Tage. Erhöhte Cholesterinwerte im Blut, auch Ichpercholesterinämie genannt, werden vom Betroffenen oft längere Zeit nicht bemerkt. In seltenen Fällen löst ein spezifischer Defekt im Erbgut erhöhte Blutfettwerte aus, dann oft schon im Kindesalter. Die Aufgabe des LDL ist es, Cholesterin aus der Leber zu den Körperzellen zu bringen. Wenn eine Lungenentzündung frühzeitig festgestellt wird, ist sie aber meist gut behandelbar und heilt in der Regel folgenlos aus. Lungenentzündungen werden meist durch Bakterien ausgelöst, insbesondere durch Pneumokopen. Pilze können vor allem bei Menschen mit starker Immunschwäche eine Lungenentzündung hervorrufen. Wenn sich eine Person mit Influenzaviren ansteckt, befallen sie die Atemwege und zerstören dort die Zellen der Schleimhaut. Die Arthrose ist eine schmerzhafte Gelenkerkrankung. Wenn man eine Grippeerkrankung überstanden hat, ist der Körper immun gegen das Influenzavirus, das die Erkrankung ausgelöst hat. Eine Lungenentzündung, auch Pneumonie genannt, ist eine plötzlich auftretende Erkrankung der unteren Atemwege. An Grippe oder Influenza erkranken vor allem in der kalten Jahreszeit vermehrt Menschen. Grippe beginnt typischerweise plötzlich mit Fieber, trockenem Husten, Halsschmerzen, Muskelschmerzen und Kopfschmerzen. Schätzungsweise zwischen 5 und 20% der Bevölkerung stecken sich jährlich in Deutschland mit Influenzaviren an. Wenn die Lunge entzündet ist, dann kann sich auch die dünne Haut um die Lunge herum das Brustfell entzünden. Unter Adipositas versteht man ein zu hohes Körpergewicht, das durch einen übermäßig hohen Fettanteil verursacht wird. Der hohe Fettanteil entsteht, wenn dem Körper, etwa durch eine kalorienreiche Ernährung, mehr Energie zugefügt wird. Körperliche Aktivität reduziert das Risiko für Herzerkrankungen, Schlaganfälle und Diabetes. Enzephalitis ist eine Entzündung des Gehirns, die durch Viren oder andere Erreger verursacht wird. Arthrose entsteht, wenn der Gelenkknorpel irreparabel geschädigt wird. Meningokokken sind Bakterien, die eine schwere Infektion der Hirnhäute oder eine Blutvergiftung verursachen können. Stress spielt bei der Entstehung einer Depression eine entscheidende Rolle. Schwere und chronische Krankheiten sind zudem eine Dauerbelastung für die Psyche. Diabetes mellitus ist eine krankhafte Störung des Zuckerstoffwechsels. Körperliche Gesundheit bedeutet, dass der körperliche Zustand eines Menschen keinen negativen Einflüssen, Einschränkungen oder Verletzungen unterliegt. Psychische Gesundheit ist ein Zustand, in dem ein Mensch keinen seelischen und geistigen Einschränkungen oder Störungen unterliegt. Eine Erkältung oder auch grippaler Infekt ist eine Erkrankung der oberen Atemwege."

Assembly AI

In [30]:
! pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 4.5 MB/s eta 0:00:00


In [31]:
import assemblyai as aai

In [32]:
aai.settings.api_key = "1e69c646087748c78b5cf5af4a94d3fc"

config = aai.TranscriptionConfig(language_code="de", speech_model= aai.SpeechModel.best)

transcriber = aai.Transcriber(config=config)
transcript = transcriber.transcribe("https://www.dropbox.com/scl/fi/crbc2mu2an7reuwzvdjml/german_audio_full.mp3?rlkey=l3wb7df1bc115oimx7hicuh3q&st=wlrnhxgf&dl=1")


In [33]:
print(transcript.text)

assembly_ai_transcript = transcript.text

Manche Krankheiten treten gehäuft zu bestimmten Jahreszeiten auf. Zwischen der Ansteckung und dem Krankheitsbeginn liegen üblicherweise 12 Stunden bis drei Tage. Eine Erkältung beginnt in vielen Fällen zunächst mit Halsschmerzen. Eine unkomplizierte Erkältung dauert normalerweise neun Tage, bei Kindern bis zu 15 Tage. Erhöhte Cholesterinwerte im Blut, auch Hypercholesterinämie genannt, werden von Betroffenen oft längere Zeit nicht bemerkt. In seltenen Fällen löst ein spezifischer Defekt im Erbgurt erhöhte Blutfettwerte aus, dann oft schon im Kindesalter. Die Aufgabe des LDL ist es, Cholesterin aus der Leber zu den Körperzellen zu bringen. Wenn eine Lungenentzündung frühzeitig festgestellt wird, ist sie aber meist gut behandelbar und heilt in der Regel folgenlos aus. Lungenentzündungen werden meist durch Bakterien ausgelöst, insbesondere durch Pneumokokken. Pilze können vor allem bei Menschen mit starker Immunschwäche eine Lungenentzündung hervorrufen. Wenn sich eine Person mit Influenz

Whipser Large from Openai

In [15]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 25.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 840.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB

In [16]:
# Load Whisper Large model
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:31<00:00, 96.9MiB/s]


In [17]:
whisper_large = model.transcribe("/content/drive/MyDrive/Audio to text/FB_PROMPT_HC_GERMAN/german_audio_full", language="de")
whisper_large["text"]

 Manche Krankheiten treten gehäuft zu bestimmten Jahreszeiten auf. Zwischen der Ansteckung und dem Krankheitsbeginn liegen üblicherweise 12 Stunden bis 3 Tage. Eine Erkältung beginnt in vielen Fällen zunächst mit Halsschmerzen. Eine unkomplizierte Erkältung dauert normalerweise 9 Tage, bei Kindern bis zu 15 Tage. Erhöhte Cholesterinwerte im Blut, auch Ichpercholesterinämie genannt, werden vom Betroffenen oft längere Zeit nicht bemerkt. In seltenen Fällen löst ein spezifischer Defekt im Erbgut erhöhte Blutfettwerte aus, dann oft schon im Kindesalter. Die Aufgabe des LDL ist es, Cholesterin auszudrücken. Der Leber zu den Körperzellen zu bringen. Wenn eine Lungenentzündung frühzeitig festgestellt wird, ist sie aber meist gut behandelbar und heilt in der Regel folgenlos aus. Lungenentzündungen werden meist durch Bakterien ausgelöst, insbesondere durch Pneumokokken. Pilze können vor allem bei Menschen mit starker Immunschwäche eine Lungenentzündung hervorrufen. Wenn sich eine Person mit Inf

Load AWS Transribe transcription - From Sami

In [34]:
# Load JSON file
with open('AWS_transcribe_transcription.json', 'r') as file:
    AWS_transcription = json.load(file)  # Load JSON as dictionary

# Extract the items (actual transcript data)
transcript_data = AWS_transcription['results']['audio_segments']

# Normalize the JSON structure into a DataFrame
transcript_AWS_df = pd.json_normalize(transcript_data)

# Display DataFrame
transcript_AWS_df.head()


,id,transcript,start_time,end_time,items
0,0,Manche Krankheiten treten gehäuft zu bestimmte...,1.649,5.769,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
1,1,Zwischen der Ansteckung und dem Krankheits Beg...,7.059,12.8,"[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20..."
2,2,Eine Erkältung beginnt in vielen Fällen zunäch...,14.319,18.129,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33]"
3,3,Eine unkomplizierte Erkältung dauert normalerw...,20.59,26.34,"[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4..."
4,4,erhöhte Cholesterinwerte in Blut. Auch ich Pop...,27.76,38.04,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 5..."


In [35]:
AWS_text = ' '.join(transcript_AWS_df['transcript'].values)

In [36]:
AWS_text

'Manche Krankheiten treten gehäuft zu bestimmten Jahreszeiten auf. Zwischen der Ansteckung und dem Krankheits Begin liegen üblicherweise 12 Stunden bis drei Tage. Eine Erkältung beginnt in vielen Fällen zunächst mit Halsschmerzen. Eine unkomplizierte Erkältung dauert normalerweise neun Tage, bei Kindern bis zu 15 Tage erhöhte Cholesterinwerte in Blut. Auch ich Populistin Nemie genannt werden vom Betroffenen oft längere Zeit nicht bemerkt. In seltenen Fällen löst ein spezifischer Defekt im Erb Gurt erhöhte Blutfettwerte aus. Dann oft schon im Kindesalter die Aufgabe des L d L is es, Cholesterin aus der Leber zu den Körper zu bringen. Wenn eine Lungenentzündung frühzeitig festgestellt wird, ist sie aber meist gut behandelbar und heilt in der Regel folgenlos aus Lungenentzündungen werden meist durch Bakterien ausgelöst, insbesondere durch Pneumokokken Pilze können vor allem bei Menschen mit starker Immunschwäche eine Lungenentzündung hervorrufen. Wenn sich eine Person mit Influenzaviren a

## Evaluation

In [37]:
!pip install Rouge jiwer

In [38]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from jiwer import wer, mer, wil
import re

# Download necessary NLTK resources
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')  # For better tokenization


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
def normalize_german_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

def german_tokenize(text):
    return nltk.word_tokenize(text, language='german')

def character_error_rate(reference, hypothesis):
    ref_chars = list(reference.lower())
    hyp_chars = list(hypothesis.lower())

    d = np.zeros((len(ref_chars) + 1, len(hyp_chars) + 1), dtype=np.int32)

    for i in range(len(ref_chars) + 1):
        d[i, 0] = i
    for j in range(len(hyp_chars) + 1):
        d[0, j] = j

    for i in range(1, len(ref_chars) + 1):
        for j in range(1, len(hyp_chars) + 1):
            if ref_chars[i-1] == hyp_chars[j-1]:
                d[i, j] = d[i-1, j-1]
            else:
                d[i, j] = min(d[i-1, j] + 1, d[i, j-1] + 1, d[i-1, j-1] + 1)

    return d[len(ref_chars), len(hyp_chars)] / len(ref_chars) if len(ref_chars) > 0 else 0

def evaluate_transcription(ground_truth, transcription, transcript_name="Model", language="german"):
    ground_truth_norm = normalize_german_text(ground_truth)
    transcription_norm = normalize_german_text(transcription)

    ground_truth_tokens = german_tokenize(ground_truth_norm)
    transcription_tokens = german_tokenize(transcription_norm)

    metrics = {'Model': transcript_name}

    smooth = SmoothingFunction().method1
    metrics['BLEU-1'] = sentence_bleu([ground_truth_tokens], transcription_tokens, weights=(1, 0, 0, 0), smoothing_function=smooth)
    metrics['BLEU-2'] = sentence_bleu([ground_truth_tokens], transcription_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=smooth)
    metrics['BLEU-3'] = sentence_bleu([ground_truth_tokens], transcription_tokens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smooth)
    metrics['BLEU-4'] = sentence_bleu([ground_truth_tokens], transcription_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smooth)

    try:
        metrics['METEOR'] = meteor_score([ground_truth_tokens], transcription_tokens)
    except:
        metrics['METEOR'] = None
        print(f"Warning: METEOR failed for {transcript_name}.")

    try:
        rouge = Rouge()
        rouge_scores = rouge.get_scores(transcription_norm, ground_truth_norm)[0]
        metrics['ROUGE-1-F'] = rouge_scores['rouge-1']['f']
        metrics['ROUGE-2-F'] = rouge_scores['rouge-2']['f']
        metrics['ROUGE-L-F'] = rouge_scores['rouge-l']['f']
    except:
        metrics['ROUGE-1-F'] = metrics['ROUGE-2-F'] = metrics['ROUGE-L-F'] = None
        print(f"Warning: ROUGE failed for {transcript_name}.")

    metrics['WER'] = wer(ground_truth, transcription)
    metrics['MER'] = mer(ground_truth, transcription)
    metrics['WIL'] = wil(ground_truth, transcription)
    metrics['CER'] = character_error_rate(ground_truth, transcription)
    metrics['Word Accuracy'] = 1 - metrics['WER']

    return metrics

def evaluate_multiple_transcriptions(ground_truth, transcriptions_dict, language="german"):
    results = []
    for model_name, transcription in transcriptions_dict.items():
        metrics = evaluate_transcription(ground_truth, transcription, model_name, language)
        results.append(metrics)

    return pd.DataFrame(results)

In [23]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [40]:
if __name__ == "__main__":
    ground_truth = text

    transcriptions = {
        "salad": salad_transcript,
        "assembly": assembly_ai_transcript,
        "whisper" : whisper_large["text"],
        "AWS Transcribe": AWS_text
    }

    results = evaluate_multiple_transcriptions(ground_truth, transcriptions)
    print(results)

    # Save results to CSV
    results.to_csv("german_transcription_evaluation_results.csv", index=False)

            Model    BLEU-1    BLEU-2    BLEU-3    BLEU-4    METEOR  \
0           salad  0.969697  0.947895  0.925671  0.903540  0.888110   
1        assembly  0.969685  0.953054  0.935964  0.917313  0.893093   
2         whisper  0.905437  0.880089  0.855737  0.830267  0.856604   
3  AWS Transcribe  0.932836  0.904902  0.876831  0.847804  0.864288   

   ROUGE-1-F  ROUGE-2-F  ROUGE-L-F     WER       MER       WIL       CER  \
0   0.957447   0.926316   0.787234  0.5025  0.411043  0.476364  0.490647   
1   0.970085   0.939314   0.803419  0.4900  0.400818  0.456652  0.487365   
2   0.941423   0.887468   0.774059  0.5725  0.444660  0.516572  0.550377   
3   0.920168   0.882507   0.756303  0.5675  0.456740  0.547767  0.498195   

   Word Accuracy  
0         0.4975  
1         0.5100  
2         0.4275  
3         0.4325  
